In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20201101.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],Rank your movie choices. You don't have to do a full ranking. [Eighth Choice],Rank your movie choices. You don't have to do a full ranking. [Ninth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice],What theme should we explore next week? [Eighth Choice],What theme should we explore next week? [Ninth Choice],What theme should we explore next week? [Tenth Choice]
0,10/27/2020 17:47:09,Bad Lieutenant: Port of Call New Orleans (2009),Raising Arizona (1987),Leaving Las Vegas (1995),Adaptation (2002),Vampire's Kiss (1988),Wild at Heart (1990),Red Rock West (1993),Bringing Out The Dead (1999),Amos & Andrew (1993),THIS HITMAN IS VERY DEPRESSED,MAYBE THE 90’S WERE PROBLEMATIC,HEIST HEIST BABY,THE MONSTER IS A METAPHOR,1970'S SCI-FI,SEXUAL AWAKENING: FRIEND OR FOE?,GADGETS,EMILIO ESTEVEZ FILM FESTIVEZ,POLITICAL THRILLRIDE,No Theme/Free-for-all
1,10/27/2020 17:57:11,Vampire's Kiss (1988),Bringing Out The Dead (1999),Bad Lieutenant: Port of Call New Orleans (2009),Adaptation (2002),Leaving Las Vegas (1995),Wild at Heart (1990),Red Rock West (1993),Raising Arizona (1987),Amos & Andrew (1993),1970'S SCI-FI,MAYBE THE 90’S WERE PROBLEMATIC,THIS HITMAN IS VERY DEPRESSED,POLITICAL THRILLRIDE,THE MONSTER IS A METAPHOR,HEIST HEIST BABY,GADGETS,EMILIO ESTEVEZ FILM FESTIVEZ,SEXUAL AWAKENING: FRIEND OR FOE?,No Theme/Free-for-all
2,10/27/2020 18:39:51,Bringing Out The Dead (1999),Raising Arizona (1987),Red Rock West (1993),Vampire's Kiss (1988),Bad Lieutenant: Port of Call New Orleans (2009),Wild at Heart (1990),Leaving Las Vegas (1995),Adaptation (2002),Amos & Andrew (1993),POLITICAL THRILLRIDE,THIS HITMAN IS VERY DEPRESSED,THE MONSTER IS A METAPHOR,GADGETS,HEIST HEIST BABY,EMILIO ESTEVEZ FILM FESTIVEZ,1970'S SCI-FI,MAYBE THE 90’S WERE PROBLEMATIC,SEXUAL AWAKENING: FRIEND OR FOE?,No Theme/Free-for-all
3,10/27/2020 18:39:57,Bringing Out The Dead (1999),Red Rock West (1993),Wild at Heart (1990),Vampire's Kiss (1988),Raising Arizona (1987),Amos & Andrew (1993),Adaptation (2002),Bad Lieutenant: Port of Call New Orleans (2009),Leaving Las Vegas (1995),MAYBE THE 90’S WERE PROBLEMATIC,SEXUAL AWAKENING: FRIEND OR FOE?,THIS HITMAN IS VERY DEPRESSED,1970'S SCI-FI,EMILIO ESTEVEZ FILM FESTIVEZ,GADGETS,HEIST HEIST BABY,THE MONSTER IS A METAPHOR,POLITICAL THRILLRIDE,No Theme/Free-for-all
4,10/27/2020 18:42:00,Wild at Heart (1990),Adaptation (2002),Red Rock West (1993),Vampire's Kiss (1988),Amos & Andrew (1993),Leaving Las Vegas (1995),Bringing Out The Dead (1999),Bad Lieutenant: Port of Call New Orleans (2009),Raising Arizona (1987),1970'S SCI-FI,THIS HITMAN IS VERY DEPRESSED,SEXUAL AWAKENING: FRIEND OR FOE?,POLITICAL THRILLRIDE,THE MONSTER IS A METAPHOR,MAYBE THE 90’S WERE PROBLEMATIC,HEIST HEIST BABY,GADGETS,EMILIO ESTEVEZ FILM FESTIVEZ,No Theme/Free-for-all
5,10/27/2020 19:24:18,Red Rock West (1993),Raising Arizona (1987),Bad Lieutenant: Port of Call New Orleans (2009),Wild at Heart (1990),Vampire's Kiss (1988),Leaving Las Vegas (1995),Amos & Andrew (1993),Bringing Out The Dead (1999),Adaptation (2002),1970'S SCI-FI,THIS HITMAN IS VERY DEPRESSED,HEIST HEIST BABY,EMILIO ESTEVEZ 

In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Bad Lieutenant: Port of Call New Orleans (2009)',
       'Raising Arizona (1987)', 'Leaving Las Vegas (1995)',
       'Adaptation (2002)', "Vampire's Kiss (1988)",
       'Wild at Heart (1990)', 'Red Rock West (1993)',
       'Bringing Out The Dead (1999)', 'Amos & Andrew (1993)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Bad Lieutenant: Port of Call New Orleans (2009)': <Candidate('Bad Lieutenant: Port of Call New Orleans (2009)')>,
 'Raising Arizona (1987)': <Candidate('Raising Arizona (1987)')>,
 'Leaving Las Vegas (1995)': <Candidate('Leaving Las Vegas (1995)')>,
 'Adaptation (2002)': <Candidate('Adaptation (2002)')>,
 "Vampire's Kiss (1988)": <Candidate('Vampire's Kiss (1988)')>,
 'Wild at Heart (1990)': <Candidate('Wild at Heart (1990)')>,
 'Red Rock West (1993)': <Candidate('Red Rock West (1993)')>,
 'Bringing Out The Dead (1999)': <Candidate('Bringing Out The Dead (1999)')>,
 'Amos & Andrew (1993)': <Candidate('Amos & Andrew (1993)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Bad Lieutenant: Port of Call New Orleans (2009), Raising Arizona (1987), Leaving Las Vegas (1995), Adaptation (2002), Vampire's Kiss (1988), Wild at Heart (1990), Red Rock West (1993), Bringing Out The Dead (1999), Amos & Andrew (1993))>,
 <Ballot(Vampire's Kiss (1988), Bringing Out The Dead (1999), Bad Lieutenant: Port of Call New Orleans (2009), Adaptation (2002), Leaving Las Vegas (1995), Wild at Heart (1990), Red Rock West (1993), Raising Arizona (1987), Amos & Andrew (1993))>,
 <Ballot(Bringing Out The Dead (1999), Raising Arizona (1987), Red Rock West (1993), Vampire's Kiss (1988), Bad Lieutenant: Port of Call New Orleans (2009), Wild at Heart (1990), Leaving Las Vegas (1995), Adaptation (2002), Amos & Andrew (1993))>,
 <Ballot(Bringing Out The Dead (1999), Red Rock West (1993), Wild at Heart (1990), Vampire's Kiss (1988), Raising Arizona (1987), Amos & Andrew (1993), Adaptation (2002), Bad Lieutenant: Port of Call New Orleans (2009), Leaving Las Vegas (1995))>,
 <Ballot

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                          Votes  Status
-----------------------------------------------  -------  --------
Bringing Out The Dead (1999)                           2  Hopeful
Vampire's Kiss (1988)                                  2  Hopeful
Wild at Heart (1990)                                   2  Hopeful
Raising Arizona (1987)                                 1  Hopeful
Red Rock West (1993)                                   1  Hopeful
Bad Lieutenant: Port of Call New Orleans (2009)        1  Hopeful
Amos & Andrew (1993)                                   1  Rejected
Adaptation (2002)                                      0  Rejected
Leaving Las Vegas (1995)                               0  Rejected

ROUND 2
Candidate                                          Votes  Status
-----------------------------------------------  -------  --------
Bringing Out The Dead (1999)                           2  Hopeful
Vampire's Kiss (1988)                                  2

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                           Votes  Status
--------------------------------  -------  --------
1970'S SCI-FI                           4  Hopeful
THIS HITMAN IS VERY DEPRESSED           2  Hopeful
HEIST HEIST BABY                        2  Hopeful
MAYBE THE 90’S WERE PROBLEMATIC         1  Rejected
POLITICAL THRILLRIDE                    1  Rejected
SEXUAL AWAKENING: FRIEND OR FOE?        0  Rejected
EMILIO ESTEVEZ FILM FESTIVEZ            0  Rejected
THE MONSTER IS A METAPHOR               0  Rejected
GADGETS                                 0  Rejected
No Theme/Free-for-all                   0  Rejected

ROUND 2
Candidate                           Votes  Status
--------------------------------  -------  --------
1970'S SCI-FI                           4  Hopeful
THIS HITMAN IS VERY DEPRESSED           4  Hopeful
HEIST HEIST BABY                        2  Rejected
MAYBE THE 90’S WERE PROBLEMATIC         0  Rejected
POLITICAL THRILLRIDE                    0  Rejected
SEXU